In [22]:
import numpy as np
import os
import tensorflow as tf

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

from sklearn.metrics import classification_report

## 1. Graph initialization

In [12]:
commands_train_dir='data/speech_data/commands/train/'
commands_test_dir='data/speech_data/commands/test/'

digits_train_dir='data/speech_data/digits/test/'
digits_test_dir='data/speech_data/digits/train/'


def load_graph(filename,name=''):
    """Unpersists graph from file as default graph."""
    with tf.gfile.FastGFile(filename, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        tf.import_graph_def(graph_def, name=name)
    return graph_def

def run_graph(sess,output_tensor,wav_file):
    with open(wav_file, 'rb') as wav_file:
        wav_data = wav_file.read()
    input_layer_name = 'wav_data:0'
    predictions, = sess.run(output_tensor, {input_layer_name: wav_data})
    return predictions        

def is_wav(path):
    _, file_extension = os.path.splitext(path)
    return file_extension.lower() in ['.wav']

tf.reset_default_graph()
graph_def=load_graph('data/speech_data/conv_actions_frozen.pb')

## 2. Create train / val for commands by extracting features from NN

In [19]:
X_train = np.empty((0, 64000))
X_test = np.empty((0, 64000))

y_train = np.empty(0)
y_test = np.empty(0)

command_labels=['_silence_','_unknown_','yes','no','up','down','left','right','on','off','stop','go']
with tf.Session() as sess:
    out_tensor = sess.graph.get_tensor_by_name('Reshape_1:0')
    for d in next(os.walk(commands_test_dir))[1]:
        for f in next(os.walk(os.path.join(commands_test_dir,d)))[2]:
            if is_wav(f):
                wav_file=os.path.join(commands_test_dir,d,f)        
                preds=run_graph(sess,out_tensor,wav_file)
                X_test = np.vstack([X_test, np.array(preds)])
                y_test = np.append(y_test, command_labels.index(d))
    for d in next(os.walk(commands_train_dir))[1]:
        for f in next(os.walk(os.path.join(commands_train_dir,d)))[2]:
            if is_wav(f):
                wav_file=os.path.join(commands_train_dir,d,f)        
                preds=run_graph(sess,out_tensor,wav_file)
                X_train = np.vstack([X_train, np.array(preds)])
                y_train = np.append(y_train, command_labels.index(d))

## 3. Use different ML methods for prediction

## KNN

In [25]:
model = KNeighborsClassifier().fit(X_train, y_train) 
print(classification_report(y_test, model.predict(X_test)))

              precision    recall  f1-score   support

         2.0       0.77      1.00      0.87        10
         3.0       0.83      0.50      0.62        10
         4.0       0.17      0.10      0.12        10
         5.0       0.55      0.60      0.57        10
         6.0       0.58      0.70      0.64        10
         7.0       1.00      0.50      0.67        10
         8.0       0.75      0.60      0.67        10
         9.0       0.60      0.30      0.40        10
        10.0       0.38      1.00      0.55         8
        11.0       0.91      1.00      0.95        10

   micro avg       0.62      0.62      0.62        98
   macro avg       0.65      0.63      0.61        98
weighted avg       0.66      0.62      0.61        98



## SVM

In [27]:
model = SVC(gamma='auto').fit(X_train, y_train) 
print(classification_report(y_test, model.predict(X_test)))

              precision    recall  f1-score   support

         2.0       0.40      1.00      0.57        10
         3.0       1.00      0.50      0.67        10
         4.0       0.20      0.10      0.13        10
         5.0       0.00      0.00      0.00        10
         6.0       0.23      0.70      0.34        10
         7.0       0.26      0.60      0.36        10
         8.0       1.00      0.20      0.33        10
         9.0       0.00      0.00      0.00        10
        10.0       0.71      0.62      0.67         8
        11.0       0.00      0.00      0.00        10

   micro avg       0.37      0.37      0.37        98
   macro avg       0.38      0.37      0.31        98
weighted avg       0.37      0.37      0.30        98



/home/xperience/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## Decision Tree

In [28]:
model = DecisionTreeClassifier().fit(X_train, y_train) 
print(classification_report(y_test, model.predict(X_test)))

              precision    recall  f1-score   support

         2.0       0.00      0.00      0.00        10
         3.0       0.14      0.40      0.21        10
         4.0       0.50      0.20      0.29        10
         5.0       0.11      0.10      0.11        10
         6.0       0.50      0.30      0.37        10
         7.0       0.27      0.30      0.29        10
         8.0       0.22      0.20      0.21        10
         9.0       0.06      0.10      0.08        10
        10.0       0.29      0.25      0.27         8
        11.0       0.33      0.10      0.15        10

   micro avg       0.19      0.19      0.19        98
   macro avg       0.24      0.20      0.20        98
weighted avg       0.24      0.19      0.20        98



## Random Forest

In [29]:
model = RandomForestClassifier(n_estimators=500).fit(X_train, y_train) 
print(classification_report(y_test, model.predict(X_test)))

              precision    recall  f1-score   support

         2.0       0.62      0.80      0.70        10
         3.0       0.88      0.70      0.78        10
         4.0       1.00      0.10      0.18        10
         5.0       0.83      0.50      0.62        10
         6.0       0.62      0.80      0.70        10
         7.0       1.00      0.70      0.82        10
         8.0       0.67      0.40      0.50        10
         9.0       0.62      0.80      0.70        10
        10.0       0.33      0.75      0.46         8
        11.0       0.77      1.00      0.87        10

   micro avg       0.65      0.65      0.65        98
   macro avg       0.73      0.66      0.63        98
weighted avg       0.74      0.65      0.64        98



## Gradient Boosting

In [30]:
model = GradientBoostingClassifier().fit(X_train, y_train) 
print(classification_report(y_test, model.predict(X_test)))

              precision    recall  f1-score   support

         2.0       0.67      0.80      0.73        10
         3.0       0.60      0.30      0.40        10
         4.0       0.00      0.00      0.00        10
         5.0       0.17      0.20      0.18        10
         6.0       0.67      0.40      0.50        10
         7.0       0.55      0.60      0.57        10
         8.0       0.08      0.10      0.09        10
         9.0       0.27      0.30      0.29        10
        10.0       0.38      0.38      0.38         8
        11.0       0.29      0.50      0.37        10

   micro avg       0.36      0.36      0.36        98
   macro avg       0.37      0.36      0.35        98
weighted avg       0.37      0.36      0.35        98



## Multilayer Perceptron

In [31]:
model = MLPClassifier().fit(X_train, y_train) 
print(classification_report(y_test, model.predict(X_test)))

              precision    recall  f1-score   support

         2.0       0.91      1.00      0.95        10
         3.0       0.90      0.90      0.90        10
         4.0       0.67      0.20      0.31        10
         5.0       0.73      0.80      0.76        10
         6.0       0.90      0.90      0.90        10
         7.0       1.00      0.70      0.82        10
         8.0       0.88      0.70      0.78        10
         9.0       0.75      0.90      0.82        10
        10.0       0.47      1.00      0.64         8
        11.0       1.00      0.90      0.95        10

   micro avg       0.80      0.80      0.80        98
   macro avg       0.82      0.80      0.78        98
weighted avg       0.83      0.80      0.79        98



## 4. Create train / val for digits by extracting features from NN

In [32]:
X_train = np.empty((0, 64000))
X_test = np.empty((0, 64000))

y_train = np.empty(0)
y_test = np.empty(0)

digits_labels=['zero','one','two','three','four','five','six','seven','eight','nine']
with tf.Session() as sess:
    out_tensor = sess.graph.get_tensor_by_name('Reshape_1:0')
    for d in next(os.walk(digits_test_dir))[1]:
        for f in next(os.walk(os.path.join(digits_test_dir,d)))[2]:
            if is_wav(f):
                wav_file=os.path.join(digits_test_dir,d,f)        
                preds=run_graph(sess,out_tensor,wav_file)
                X_test = np.vstack([X_test, np.array(preds)])
                y_test = np.append(y_test, digits_labels.index(d))
    for d in next(os.walk(digits_train_dir))[1]:
        for f in next(os.walk(os.path.join(digits_train_dir,d)))[2]:
            if is_wav(f):
                wav_file=os.path.join(digits_train_dir,d,f)        
                preds=run_graph(sess,out_tensor,wav_file)
                X_train = np.vstack([X_train, np.array(preds)])
                y_train = np.append(y_train, digits_labels.index(d))

## 5. Use different ML methods for prediction

## KNN

In [33]:
model = KNeighborsClassifier().fit(X_train, y_train) 
print(classification_report(y_test, model.predict(X_test)))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00         6
         1.0       0.62      0.71      0.67         7
         2.0       0.86      0.86      0.86         7
         3.0       0.88      1.00      0.93         7
         4.0       0.75      0.86      0.80         7
         5.0       1.00      0.71      0.83         7
         6.0       0.75      0.60      0.67         5
         7.0       0.60      0.60      0.60         5
         8.0       0.78      1.00      0.88         7
         9.0       1.00      0.71      0.83         7

   micro avg       0.82      0.82      0.82        65
   macro avg       0.82      0.81      0.81        65
weighted avg       0.83      0.82      0.81        65



## SVM

In [34]:
model = SVC(gamma='auto').fit(X_train, y_train) 
print(classification_report(y_test, model.predict(X_test)))

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         6
         1.0       0.86      0.86      0.86         7
         2.0       0.39      1.00      0.56         7
         3.0       0.70      1.00      0.82         7
         4.0       0.83      0.71      0.77         7
         5.0       0.83      0.71      0.77         7
         6.0       0.00      0.00      0.00         5
         7.0       0.00      0.00      0.00         5
         8.0       0.64      1.00      0.78         7
         9.0       0.71      0.71      0.71         7

   micro avg       0.65      0.65      0.65        65
   macro avg       0.50      0.60      0.53        65
weighted avg       0.53      0.65      0.57        65



/home/xperience/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## Decision Tree

In [35]:
model = DecisionTreeClassifier().fit(X_train, y_train) 
print(classification_report(y_test, model.predict(X_test)))

              precision    recall  f1-score   support

         0.0       0.38      0.50      0.43         6
         1.0       1.00      0.57      0.73         7
         2.0       0.25      0.14      0.18         7
         3.0       0.56      0.71      0.63         7
         4.0       0.56      0.71      0.63         7
         5.0       0.44      0.57      0.50         7
         6.0       0.00      0.00      0.00         5
         7.0       0.25      0.20      0.22         5
         8.0       0.83      0.71      0.77         7
         9.0       0.56      0.71      0.63         7

   micro avg       0.51      0.51      0.51        65
   macro avg       0.48      0.48      0.47        65
weighted avg       0.51      0.51      0.49        65



## Random Forest

In [36]:
model = RandomForestClassifier(n_estimators=500).fit(X_train, y_train) 
print(classification_report(y_test, model.predict(X_test)))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00         6
         1.0       1.00      0.71      0.83         7
         2.0       0.58      1.00      0.74         7
         3.0       0.75      0.86      0.80         7
         4.0       0.75      0.86      0.80         7
         5.0       1.00      0.86      0.92         7
         6.0       0.00      0.00      0.00         5
         7.0       0.83      1.00      0.91         5
         8.0       1.00      1.00      1.00         7
         9.0       0.86      0.86      0.86         7

   micro avg       0.83      0.83      0.83        65
   macro avg       0.78      0.81      0.79        65
weighted avg       0.80      0.83      0.80        65



/home/xperience/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## Gradient Boosting

In [37]:
model = GradientBoostingClassifier().fit(X_train, y_train) 
print(classification_report(y_test, model.predict(X_test)))

              precision    recall  f1-score   support

         0.0       0.80      0.67      0.73         6
         1.0       0.50      0.43      0.46         7
         2.0       0.50      0.57      0.53         7
         3.0       0.44      0.57      0.50         7
         4.0       0.42      0.71      0.53         7
         5.0       0.67      0.57      0.62         7
         6.0       0.50      0.20      0.29         5
         7.0       0.67      0.40      0.50         5
         8.0       0.80      0.57      0.67         7
         9.0       0.67      0.86      0.75         7

   micro avg       0.57      0.57      0.57        65
   macro avg       0.60      0.56      0.56        65
weighted avg       0.59      0.57      0.56        65



## Multilayer Perceptron

In [38]:
model = MLPClassifier().fit(X_train, y_train) 
print(classification_report(y_test, model.predict(X_test)))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00         6
         1.0       0.75      0.86      0.80         7
         2.0       0.50      0.86      0.63         7
         3.0       1.00      0.86      0.92         7
         4.0       0.70      1.00      0.82         7
         5.0       1.00      0.71      0.83         7
         6.0       0.00      0.00      0.00         5
         7.0       0.83      1.00      0.91         5
         8.0       1.00      0.71      0.83         7
         9.0       1.00      1.00      1.00         7

   micro avg       0.82      0.82      0.82        65
   macro avg       0.78      0.80      0.78        65
weighted avg       0.80      0.82      0.79        65



/home/xperience/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# F1-score

https://docs.google.com/spreadsheets/d/1QK6lDiLoPQTDUYszxRFyCQdj48kLSQQQxH4TOhn-rnI/edit?usp=sharing

|  method | commands | digits |
| --- | --- | --- |
|  KNN | 0,61 | **0,81** |
|  SVM | 0,31 | 0,53 |
|  Decision Tree | 0,20 | 0,47 |
|  Random Forest | 0,63 | 0,79 |
|  Gradient Boosting | 0,35 | 0,56 |
|  MLP | **0,78** | 0,78 |